<a href="https://colab.research.google.com/github/jromerooo2/feature_extraction/blob/main/Video_Features.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Deep Features using TimeSformer and mmaction2:


In [ ]:
! pip install -U openmim
!mim install mmengine mmcv
!git clone https://github.com/open-mmlab/mmaction2.git
%cd mmaction2
!pip install -v -e .

In [34]:
from mmaction.apis import inference_recognizer, init_recognizer

config_path = 'configs/recognition/timesformer/timesformer_jointST_8xb8-8x32x1-15e_kinetics400-rgb.py'
checkpoint_path = 'https://download.openmmlab.com/mmaction/recognition/timesformer/vit_base_patch16_224.pth' # can be a local path
img_path = 'demo/demo.mp4'

model = init_recognizer(config_path, checkpoint_path, device="cpu")
result = inference_recognizer(model, img_path)
print(result)

Loads checkpoint by http backend from path: https://download.openmmlab.com/mmaction/recognition/timesformer/vit_base_patch16_224.pth
The model and loaded state dict do not match exactly

unexpected key in source state_dict: cls_token, pos_embed, patch_embed.projection.weight, patch_embed.projection.bias, transformer_layers.layers.0.norms.0.weight, transformer_layers.layers.0.norms.0.bias, transformer_layers.layers.0.attentions.0.attn.in_proj_weight, transformer_layers.layers.0.attentions.0.attn.in_proj_bias, transformer_layers.layers.0.attentions.0.attn.out_proj.weight, transformer_layers.layers.0.attentions.0.attn.out_proj.bias, transformer_layers.layers.0.norms.1.weight, transformer_layers.layers.0.norms.1.bias, transformer_layers.layers.0.ffns.0.layers.0.0.weight, transformer_layers.layers.0.ffns.0.layers.0.0.bias, transformer_layers.layers.0.ffns.0.layers.1.weight, transformer_layers.layers.0.ffns.0.layers.1.bias, transformer_layers.layers.1.norms.0.weight, transformer_layers.layer

# Handcrafted features using DLib

In [6]:
!pip install dlib

In [9]:
import sys
import os
import dlib
import glob

if len(sys.argv) != 3:
    print(
        "Give the path to the trained shape predictor model as the first "
        "argument and then the directory containing the facial images.\n"
        "For example, if you are in the python_examples folder then "
        "execute this program by running:\n"
        "    ./face_landmark_detection.py shape_predictor_68_face_landmarks.dat ../examples/faces\n"
        "You can download a trained facial shape predictor from:\n"
        "    http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2")
    exit()

predictor_path = sys.argv[1]
faces_folder_path = sys.argv[2]

detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor(predictor_path)
win = dlib.image_window()

for f in glob.glob(os.path.join(faces_folder_path, "*.jpg")):
    print("Processing file: {}".format(f))
    img = dlib.load_rgb_image(f)

    win.clear_overlay()
    win.set_image(img)

    # Ask the detector to find the bounding boxes of each face. The 1 in the
    # second argument indicates that we should upsample the image 1 time. This
    # will make everything bigger and allow us to detect more faces.
    dets = detector(img, 1)
    print("Number of faces detected: {}".format(len(dets)))
    for k, d in enumerate(dets):
        print("Detection {}: Left: {} Top: {} Right: {} Bottom: {}".format(
            k, d.left(), d.top(), d.right(), d.bottom()))
        # Get the landmarks/parts for the face in box d.
        shape = predictor(img, d)
        print("Part 0: {}, Part 1: {} ...".format(shape.part(0),
                                                  shape.part(1)))
        # Draw the face landmarks on the screen.
        win.add_overlay(shape)

    win.add_overlay(dets)
    dlib.hit_enter_to_continue()

RuntimeError: ignored